In [18]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [19]:
from datasets import load_dataset

dataset  = load_dataset("csv", data_files="/content/train.csv", split="train")
test_ds = load_dataset("csv", data_files="/content/test.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
dataset

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target'],
    num_rows: 7613
})

In [21]:
dataset["text"][1690]

"Worlds Collide When an American Family Takes Over Britain's Isle of Man in New TLC Show Suddenly Royal http://t.co/OmB3oS54tN via @People"

In [22]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [23]:
train_df.head(30)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [24]:
train_df["text"][7610]

'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ'

Form the table above, the data has tweets that have hashtags(#), tags(@), URLs, Time presenations in numerical form and emojis

The hashtags provide keywords and locations. ie. earthquates and Alaska. However locations does not seem to affect the data much. it will be dropped moving forward.

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


All samples are labeled. Not all have keywords and locations

In [26]:
import matplotlib.pyplot as plt

train_df["target"].value_counts()

,count
target,
0,4342
1,3271


The non-threat tweets are moer than the threat tweets. But not a big margin.

# Preprocessing

## Data Cleaning

The goal is to:
- Remove stopwords, punctuations, special characters, hashtags, mentions, digits, emojis and urls.
- Standardize case and accent characters.
- Expand contractions
- Lematize the words to their root form

Necessary functions will be created below.

In [27]:
# To turn texts to lower case

def to_lowercase(text):
    std_text = text.lower()
    return std_text

In [28]:
dataset["text"][24]

'LOOOOOOL'

In [29]:
to_lowercase(dataset["text"][24])

'looooool'

In [30]:
# Standardizes accent characters

import unicodedata

def standardize_accent_chars(text):
    std_text = unicodedata.normalize(
        'NFKD', text).encode(
        'ascii', 'ignore').decode(
        'utf8', 'ignore')
    return std_text

In [31]:
dataset["text"][53]

'Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze http://t.co/pcXarbH9An'

In [32]:
standardize_accent_chars(dataset["text"][53])

'Police: Arsonist Deliberately Set Black Church In North CarolinaaEAblaze http://t.co/pcXarbH9An'

In [33]:
# Removing URLs and html tags
import re

def remove_url_and_html_tags(text):
    # Remove URLs (both http and https)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Match both http and https URLs
    # Remove HTML tags
    text = re.sub(r'<.*?>+', '', text)
    return text

In [34]:
remove_url_and_html_tags(dataset["text"][53])

'Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze '

In [35]:
# Removing mentions and hashtags

def remove_mentions(text):
    std_text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove mentions
    return std_text.strip()

In [36]:
mentions =dataset["text"][7578]
mentions

'@jt_ruff23 @cameronhacker and I wrecked you both'

In [37]:
remove_mentions(mentions)

'and I wrecked you both'

Samples like these dicourage me from removing hashtags as it holds information on flooding which is a disaster. They do act as keywords.

In [38]:
# removing special characters and digits

def remove_special_characters_and_digits(text):
    pattern = r"[^a-zA-Z.,!?/:;\"\'\s]"
    std_text = re.sub(pattern, '', text)
    return std_text

In [39]:
remove_special_characters_and_digits(mentions)

'jtruff cameronhacker and I wrecked you both'

Much better in this case. the information is retained and not removed by the hashtags.

In [40]:
# Removing punctuations
import string

def remove_punctuations(text):
    return "".join([c for c in text if c not in string.punctuation])


In [41]:
punc = dataset["text"][15]
punc

"What's up man?"

In [42]:
remove_punctuations(punc)

'Whats up man'

In [43]:
# Function to clean data

def clean_data_for_mapping(batch, text_column):
    original_text = batch[text_column]
    # Apply cleaning functions sequentially
    cleaned_text = to_lowercase(original_text)
    cleaned_text = standardize_accent_chars(cleaned_text)
    cleaned_text = remove_url_and_html_tags(cleaned_text)
    cleaned_text = remove_mentions(cleaned_text)
    cleaned_text = remove_special_characters_and_digits(cleaned_text)
    cleaned_text = remove_punctuations(cleaned_text)

    # Update the batch with the cleaned text
    batch[text_column]= cleaned_text

    return batch

In [44]:
def clean_dataset(dataset, text_column="text"):
    # Apply cleaning to the dataset
    cleaned_dataset = dataset.map(
        lambda x: clean_data_for_mapping(x, text_column),
        desc="Cleaning text data"
    )
    return cleaned_dataset

In [45]:
cleaned_train_data = clean_dataset(dataset)

Cleaning text data:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [46]:
cleaned_train_data

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target'],
    num_rows: 7613
})

In [47]:
cleaned_train_data["text"][:20]

['our deeds are the reason of this earthquake may allah forgive us all',
 'forest fire near la ronge sask canada',
 'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
 ' people receive wildfires evacuation orders in california',
 'just got sent this photo from ruby alaska as smoke from wildfires pours into a school',
 'rockyfire update  california hwy  closed in both directions due to lake county fire  cafire wildfires',
 'flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas',
 'im on top of the hill and i can see a fire in the woods',
 'theres an emergency evacuation happening now in the building across the street',
 'im afraid that the tornado is coming to our area',
 'three people died from the heat wave so far',
 'haha south tampa is getting flooded hah wait a second i live in south tampa what am i gonna do what am i gonna do fvck flooding',
 'raining flooding 

In [48]:
cleaned_test_data = clean_dataset(test_ds)

Cleaning text data:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [49]:
cleaned_test_data["text"][:20]

['just happened a terrible car crash',
 'heard about earthquake is different cities stay safe everyone',
 'there is a forest fire at spot pond geese are fleeing across the street i cannot save them all',
 'apocalypse lighting spokane wildfires',
 'typhoon soudelor kills  in china and taiwan',
 'were shakingits an earthquake',
 'theyd probably still show more life than arsenal did yesterday eh eh',
 'hey how are you',
 'what a nice hat',
 'fuck off',
 'no i dont like cold',
 'nooooooooo dont do that',
 'no dont tell me that',
 'what if',
 'awesome',
 'birmingham wholesale market is ablaze bbc news  fire breaks out at birminghams wholesale market',
 'will you wear shorts for race ablaze ',
 'previouslyondoyintv toke makinwauas marriage crisis sets nigerian twitter ablaze',
 'check these out     nsfw',
 'psa iuam splitting my personalities\n\n techies follow \n burners follow']

## Tokenization

Mode to be used is DistilBERT

In [50]:
from transformers import AutoTokenizer

model = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [75]:
from huggingface_hub import notebook_login

notebook_login()

In [52]:
tokenizer.vocab_size

30522

In [53]:
tokenizer.model_max_length

512

In [54]:
tokenizer.model_input_names


['input_ids', 'token_type_ids', 'attention_mask']

In [55]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [56]:
train_tokens = dataset.map(tokenize, batched=True, batch_size=None)


Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [57]:
train_tokens

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7613
})

In [58]:
test_tokens = cleaned_test_data.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [59]:
test_tokens

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3263
})

Renaming the trarget column to labels so that distil bert can identify it.

In [60]:
train_tokens = train_tokens.rename_column("target", "label")

## Training the Classifier Model

In [61]:
from transformers import AutoModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_model = AutoModel.from_pretrained(model).to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [62]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items()
             if k in tokenizer.model_input_names}

    with torch.no_grad():
        last_hidden_state = pre_model(**inputs).last_hidden_state
        # Return vector for [CLS] token
        return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}


converting the columbs to torch formart, tensors

In [63]:
train_tokens.set_format(
"torch", columns=[
    "input_ids","attention_mask", "label", "keyword", "location"
])

extracting the hidden states across all splits in one go:

In [64]:
test_tokens.set_format(
"torch", columns=[
    "input_ids", "attention_mask", "keyword", "location"
])

In [65]:
hidden_data = train_tokens.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [66]:
hidden_test = test_tokens.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [67]:
hidden_train = hidden_data

In [68]:
hidden_train.column_names

['id',
 'keyword',
 'location',
 'text',
 'label',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [145]:
train_test_split = hidden_data.train_test_split(test_size=0.2, seed=42)

hidden_train = train_test_split['train']
hidden_val = train_test_split['test']

In [139]:
# hidden_train = hidden_train.remove_columns(["keyword", "location"])
# hidden_val = hidden_val.remove_columns(["keyword", "location"])
# hidden_test = hidden_test.remove_columns(["keyword", "location"])

### Creating a feature matrix

In [146]:
hidden_test.column_names

['id', 'text', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state']

In [147]:
hidden_val.column_names

['id',
 'keyword',
 'location',
 'text',
 'label',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [142]:
import numpy as np

X_train = np.array(hidden_train["hidden_state"])
X_valid = np.array(hidden_val["hidden_state"])
y_train = np.array(hidden_train["label"])
y_valid = np.array(hidden_val["label"])
X_train.shape, X_valid.shape



((6090,), (1523,))

### Visualizing the training set

## Fine Tuning a pretrained model

In [148]:
from transformers import AutoModelForSequenceClassification
model_2 = "google-bert/bert-base-uncased"
num_labels = 2
fine_model = (
AutoModelForSequenceClassification.from_pretrained(model_2, num_labels=num_labels).to(device))

### Defining performance metrics

In [149]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)


    return {"accuracy": acc, "f1": f1}

In [176]:
from transformers import Trainer, TrainingArguments
batch_size = 64
logging_steps = len(hidden_data) // batch_size
model_name = f"{model_2}-finetuned-disaster-NLP"
training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True,
    log_level="error"
  )



In [177]:
from transformers import Trainer
trainer = Trainer(model=fine_model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=hidden_train,
                  eval_dataset=hidden_val,
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.447759,0.836507,0.836323
2,0.284900,0.449315,0.826001,0.826116
3,0.247500,0.486300,0.824032,0.823852


TrainOutput(global_step=288, training_loss=0.25661417841911316, metrics={'train_runtime': 283.7339, 'train_samples_per_second': 64.391, 'train_steps_per_second': 1.015, 'total_flos': 788654832890400.0, 'train_loss': 0.25661417841911316, 'epoch': 3.0})

Training with 2 epochs has proved to be just enough.
Dropping the keyword and location columns reduced accuracy

In [178]:
hidden_val

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state', 'loss', 'predicted_label'],
    num_rows: 1523
})

In [179]:
preds_output = trainer.predict(hidden_val)


In [180]:
preds_output.predictions


array([[-2.2526479 ,  2.5178556 ],
       [ 1.4769304 , -1.5933392 ],
       [-1.8348513 ,  2.1783984 ],
       ...,
       [ 1.379384  , -1.6368513 ],
       [-1.7048454 ,  2.074252  ],
       [-0.32105443, -0.33468622]], dtype=float32)

In [181]:

preds_output.metrics

{'test_loss': 0.4862995147705078,
 'test_accuracy': 0.8240315167432699,
 'test_f1': 0.823852144694806,
 'test_runtime': 7.2761,
 'test_samples_per_second': 209.316,
 'test_steps_per_second': 3.298}

In [182]:
y_preds = np.argmax(preds_output.predictions, axis=1)

### Error Analysis

In [190]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    inputs = {
        k:v.to(device) for k, v in batch.items()
        if k in tokenizer.model_input_names
    }
    with torch.no_grad():
        output = fine_model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                            reduction="none")

        return {
            "loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()
        }

In [191]:
hidden_train = hidden_train.map(
forward_pass_with_label, batched=True, batch_size=16)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [185]:
hidden_val = hidden_val.map(
forward_pass_with_label, batched=True, batch_size=16)

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

AttributeError: 'Series' object has no attribute 'to'

In [ ]:
hidden_val

In [ ]:
def label_int2str(label):
    label_mapping = {0: 'non-disaster', 1: 'disaster'}
    return label_mapping.get(label, 'unknown')  # Default to 'unknown' if label is not found

In [ ]:
hidden_train.set_format("pandas")
cols = ["id", "text", "label", "keyword", "location", "predicted_label", "loss"]
hidden_val.set_format("pandas")
df_val = hidden_val[:][cols]
df_val["label"] = df_val["label"].apply(label_int2str)
df_val["predicted_label"] = (
df_val["predicted_label"].apply(label_int2str))

Gertting the samples with the highest losses

In [ ]:
df_val.sort_values("loss", ascending=False).head(10)


In [ ]:
df_val["text"][1315]

The keywords may be misleading the model and this makes teh model undsertand that there is a disaster whe there is none. ie. the arson burns a flag and the disstaer keyword in example 1063, is an averted disaster that should not be flagged as disaster.

The model also seems to now have very high losses as the highest loss is 3.0 which is less than half.

The non-disaster samples are the on that are most misrepresented

To check the predictions with the smallest losses:

In [ ]:
df_val.sort_values("loss", ascending=True).head(10)


In [ ]:
df_val["text"][249]

The models is most confident when labeling disaster samples than non-disaster samples

In [ ]:
sample_submission = pd.read_csv("/content/sample_submission.csv")

In [ ]:
hidden_test

In [ ]:
preds = trainer.predict(hidden_test)

In [ ]:
preds

In [ ]:
pred = preds.predictions.argmax(axis=-1)  # Keep original pred calculation
predictions_df = pd.DataFrame(pred, columns=["predicted_label"]) # Create a new DataFrame to store predictions with the desired column name

In [ ]:
predictions_df["predicted_label"]

In [ ]:
sample_submission["target"] = predictions_df["predicted_label"]

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv("submission.csv", index=False)
